In [151]:
from neuromaps.datasets import available_annotations
from neuromaps.datasets import fetch_annotation
from neuromaps.parcellate import Parcellater
from netneurotools import datasets as nntdata
from neuromaps.images import construct_shape_gii, load_gifti, load_data
from neuromaps import images
from neuromaps.stats import compare_images
from neuromaps import transforms
import json
import nibabel as nib
import numpy as np
import os
from os.path import join as ospj
import re
import pandas as pd
import sys
 

In [167]:
# load aggregated age map

agg = pd.read_csv("/cbica/projects/luo_wm_dev/output/HBN/tract_profiles/tract_to_cortex/HBN_agg_ageeffects_ordered_all.csv")

In [3]:
agg

,Unnamed: 0,region,regional_mean_ageeffect,coverage,regionName,regionLongName,regionIdLabel,LR,Lobe,cortex,regionID,Cortex_ID,x.cog,y.cog,z.cog,volmm,label
0,1,V1,0.223275,yes,V1_R,Primary_Visual_Cortex_R,201_R,R,Occ,Primary_Visual,181,1,78.060375,44.539286,74.333474,7089,R_V1_ROI
1,2,MST,0.278588,yes,MST_R,Medial_Superior_Temporal_Area_R,202_R,R,Occ,MT+_Complex_and_Neighboring_Visual_Areas,182,5,43.620295,63.751227,78.013093,611,R_MST_ROI
2,3,V6,0.268083,yes,V6_R,Sixth_Visual_Area_R,203_R,R,Occ,Dorsal_Stream_Visual,183,3,72.226868,49.256228,102.929715,1124,R_V6_ROI
3,4,V2,0.191993,yes,V2_R,Second_Visual_Area_R,204_R,R,Occ,Early_Visual,184,2,77.026083,47.558042,76.305265,6211,R_V2_ROI
4,5,V3,0.260230,yes,V3_R,Third_Visual_Area_R,205_R,R,Occ,Early_Visual,185,2,71.976821,41.580308,78.859282,4875,R_V3_ROI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,356,STSva,0.152273,yes,STSva_L,Area_STSv_anterior_L,176_L,L,Temp,Auditory_Association,176,11,142.689243,118.134462,52.832669,1004,L_STSva_ROI
356,357,TE1m,0.313459,yes,TE1m_L,Area_TE1_Middle_L,177_L,L,Temp,Lateral_Temporal,177,14,155.554922,101.395287,54.901178,2631,L_TE1m_ROI
357,358,PI,0.212804,yes,PI_L,Para-Insular_Area_L,178_L,L,Temp,Insular_and_Frontal_Opercular,178,12,134.650549,124.293407,56.102198,910,L_PI_ROI
358,359,a32pr,0.292845,yes,a32pr_L,Area_anterior_32_prime_L,179_L,L,Fr,Anterior_Cingulate_and_Medial_Prefrontal,179,19,97.782313,155.883735,101.539270,1617,L_a32pr_ROI


In [4]:
for annotation in available_annotations('raichle'):
    print(annotation)

('raichle', 'cbf', 'fsLR', '164k')
('raichle', 'cbv', 'fsLR', '164k')
('raichle', 'cmr02', 'fsLR', '164k')
('raichle', 'cmruglu', 'fsLR', '164k')


In [16]:
for annotation in available_annotations('satterthwaite2014'):
    print(annotation)

('satterthwaite2014', 'meancbf', 'MNI152', '1mm')


In [17]:
for annotation in available_annotations('hcps1200'):
    print(annotation)

('hcps1200', 'megalpha', 'fsLR', '4k')
('hcps1200', 'megbeta', 'fsLR', '4k')
('hcps1200', 'megdelta', 'fsLR', '4k')
('hcps1200', 'meggamma1', 'fsLR', '4k')
('hcps1200', 'meggamma2', 'fsLR', '4k')
('hcps1200', 'megtheta', 'fsLR', '4k')
('hcps1200', 'megtimescale', 'fsLR', '4k')
('hcps1200', 'myelinmap', 'fsLR', '32k')
('hcps1200', 'thickness', 'fsLR', '32k')


In [57]:

for annotation in available_annotations('beliveau2017'):
    print(annotation)

('beliveau2017', 'az10419369', 'MNI152', '1mm')
('beliveau2017', 'az10419369', 'fsaverage', '164k')
('beliveau2017', 'cimbi36', 'MNI152', '1mm')
('beliveau2017', 'cimbi36', 'fsaverage', '164k')
('beliveau2017', 'cumi101', 'MNI152', '1mm')
('beliveau2017', 'cumi101', 'fsaverage', '164k')
('beliveau2017', 'dasb', 'MNI152', '1mm')
('beliveau2017', 'dasb', 'fsaverage', '164k')
('beliveau2017', 'sb207145', 'MNI152', '1mm')
('beliveau2017', 'sb207145', 'fsaverage', '164k')


In [106]:
raichle = fetch_annotation(source=['raichle'])
reardon = fetch_annotation(source=['reardon2018'])
ted = fetch_annotation(source=['satterthwaite2014'])
hcps = fetch_annotation(source=['hcps1200'])
beliveau = fetch_annotation(source=['beliveau2017'])

genes = fetch_annotation(source='abagen', desc='genepc1')
hill = fetch_annotation(source = 'hill2010', desc = 'devexp')


/Users/audluo/anaconda3/envs/luo_wm_dev/lib/python3.8/site-packages/neuromaps/datasets/annotations.py:238: UserWarning: Data from beliveau2017 and norgaard2021 is best used in the provided fsaverage space (e.g. source='beliveau2017', space='fsaverage', den='164k'). MNI152 maps should only be used for subcortical data.
  warnings.warn('Data from beliveau2017 and norgaard2021 is best used in'


- need to transform surfaces to fslr 32k
- then parcellate the maps to glasser
- then compare the brain maps
- i'm lazy

to subset: 
`hcps[('hcps1200', 'megbeta', 'fsLR', '4k')]`

In [4]:
#########################################################
# Load glasser
#########################################################
lh_glasser = nib.load("/cbica/projects/luo_wm_dev/atlases/glasser/FSLRVertex/glasser_360_L.label.gii")
rh_glasser = nib.load("/cbica/projects/luo_wm_dev/atlases/glasser/FSLRVertex/glasser_360_R.label.gii")

# generate neuromaps fsLR based Glasser 360 parcellation (needs relabeling to have consecutive region IDs)
glasser = images.relabel_gifti((lh_glasser, rh_glasser), background=['Medial_wall'])

# let's assign these labels so that we can call them later for plotting
lh_glasser = glasser[0]
rh_glasser = glasser[1]

# create parcellater object
glasser_parc = Parcellater(glasser, 'fsLR').fit()
print(glasser_parc)

In [48]:
hcps[('hcps1200', 'megalpha', 'fsLR', '4k')][1]
hcps[('hcps1200', 'thickness', 'fsLR', '32k')][1]

'/Users/audluo/neuromaps-data/annotations/hcps1200/thickness/fsLR/source-hcps1200_desc-thickness_space-fsLR_den-32k_hemi-R_feature.func.gii'

In [128]:

hcps_parc = {}

for key, value in hcps.items():
    left_gii = value[0]  
    right_gii = value[1]  
    combined_gii = [left_gii, right_gii]
    transformed_values = glasser_parc.fit_transform(combined_gii, 'fsLR')
    hcps_parc[key] = transformed_values

In [9]:
hcps_parc.keys()

dict_keys([('hcps1200', 'megalpha', 'fsLR', '4k'), ('hcps1200', 'megbeta', 'fsLR', '4k'), ('hcps1200', 'megdelta', 'fsLR', '4k'), ('hcps1200', 'meggamma1', 'fsLR', '4k'), ('hcps1200', 'meggamma2', 'fsLR', '4k'), ('hcps1200', 'megtheta', 'fsLR', '4k'), ('hcps1200', 'megtimescale', 'fsLR', '4k'), ('hcps1200', 'myelinmap', 'fsLR', '32k'), ('hcps1200', 'thickness', 'fsLR', '32k')])

In [168]:
from neuromaps import datasets, images, nulls, resampling

rotated = nulls.alexander_bloch(
    data=agg['regional_mean_ageeffect'],
    atlas='fsLR',                 # Specify atlas as fsLR
    density='32k',                # Choose density matching your data, e.g., '32k'
    parcellation=(lh_glasser, rh_glasser),  # Pass the Glasser parcellation tuple
    n_perm=1000,                  # Number of permutations
    seed=42                       # Optional seed for reproducibility
)

In [169]:

compare_images(hcps_parc[('hcps1200', 'megalpha', 'fsLR', '4k')], agg['regional_mean_ageeffect'], metric='pearsonr', nulls=rotated)

(0.20406639993287454, 0.6433566433566433)

In [170]:
compare_images(hcps_parc[('hcps1200', 'megbeta', 'fsLR', '4k')], agg['regional_mean_ageeffect'], metric='pearsonr', nulls=rotated)

(0.5458672575958705, 0.058941058941058944)

In [173]:
compare_images(hcps_parc[('hcps1200', 'megdelta', 'fsLR', '4k')], agg['regional_mean_ageeffect'], metric='pearsonr', nulls = rotated)

(-0.4649153331868147, 0.14685314685314685)

In [174]:
compare_images(hcps_parc[('hcps1200', 'meggamma1', 'fsLR', '4k')], agg['regional_mean_ageeffect'], metric='pearsonr')

-0.15497817745302697

In [127]:
hcps_parc.keys()

dict_keys([('hcps1200', 'megalpha', 'fsLR', '4k'), ('hcps1200', 'megbeta', 'fsLR', '4k'), ('hcps1200', 'megdelta', 'fsLR', '4k'), ('hcps1200', 'meggamma1', 'fsLR', '4k')])

In [176]:
compare_images(hcps_parc[('hcps1200', 'meggamma2', 'fsLR', '4k')], agg['regional_mean_ageeffect'], metric='pearsonr', nulls=rotated)

(-0.4834890746450803, 0.12287712287712288)

In [177]:
compare_images(hcps_parc[('hcps1200', 'megtheta', 'fsLR', '4k')], agg['regional_mean_ageeffect'], metric='pearsonr', nulls=rotated)

(0.02788659426918444, 0.955044955044955)

In [178]:
compare_images(hcps_parc[('hcps1200', 'megtimescale', 'fsLR', '4k')], agg['regional_mean_ageeffect'], metric='pearsonr', nulls=rotated)

(-0.47787245545750284, 0.13086913086913088)

In [179]:
compare_images(hcps_parc[('hcps1200', 'myelinmap', 'fsLR', '32k')], agg['regional_mean_ageeffect'], metric='pearsonr')

0.34194770216212755

In [181]:
compare_images(hcps_parc[('hcps1200', 'thickness', 'fsLR', '32k')], agg['regional_mean_ageeffect'], metric='pearsonr', nulls=rotated)

(-0.4521002826916679, 0.16783216783216784)

In [182]:
ted_fslr = transforms.mni152_to_fslr(ted, '32k')

In [32]:
#ted_fslr

left_gii = ted_fslr[0]  
right_gii = ted_fslr[1]  
combined_gii = [left_gii, right_gii]
transformed_values = glasser_parc.fit_transform(combined_gii, 'fsLR')
ted_parc = transformed_values

In [183]:
compare_images(ted_parc, agg['regional_mean_ageeffect'], metric='pearsonr')

-0.2455570048542545

In [35]:

raichle_parc = {}

for key, value in raichle.items():
    left_gii = value[0]  
    right_gii = value[1]  
    combined_gii = [left_gii, right_gii]
    transformed_values = glasser_parc.fit_transform(combined_gii, 'fsLR')
    raichle_parc[key] = transformed_values

Downloaded 15056896 of 28260568 bytes (53.3%,    0.9s remaining) ...done. (3 seconds, 0 min)
Extracting data from /Users/audluo/neuromaps-data/d9a7bf9d3cbc8ffc454fc7f993cf94ae/fsLR164k.tar.gz..... done.


In [36]:
raichle.keys()

dict_keys([('raichle', 'cbf', 'fsLR', '164k'), ('raichle', 'cbv', 'fsLR', '164k'), ('raichle', 'cmr02', 'fsLR', '164k'), ('raichle', 'cmruglu', 'fsLR', '164k')])

In [184]:
compare_images(raichle_parc[('raichle', 'cbf', 'fsLR', '164k')], agg['regional_mean_ageeffect'], metric='pearsonr', nulls=rotated)

(0.15894762275507895, 0.7222777222777222)

In [185]:
compare_images(raichle_parc[('raichle', 'cbv', 'fsLR', '164k')], agg['regional_mean_ageeffect'], metric='pearsonr')

-0.02395888996646853

In [186]:
compare_images(raichle_parc[('raichle', 'cmr02', 'fsLR', '164k')], agg['regional_mean_ageeffect'], metric='pearsonr', nulls = rotated)

(0.3230414583425527, 0.3936063936063936)

In [187]:
compare_images(raichle_parc[('raichle', 'cmruglu', 'fsLR', '164k')], agg['regional_mean_ageeffect'], metric='pearsonr', nulls=rotated)

(0.3031155543826755, 0.43956043956043955)

In [65]:
beliveau_fslr = {}


for key, value in beliveau.items():
    if 'mni' in key[2].lower():  # Check if 'mni' is part of the key
        print(key)
        beliveau_fslr[key] = transforms.mni152_to_fslr(beliveau[key], '32k')




('beliveau2017', 'az10419369', 'MNI152', '1mm')
('beliveau2017', 'cimbi36', 'MNI152', '1mm')
('beliveau2017', 'cumi101', 'MNI152', '1mm')
('beliveau2017', 'dasb', 'MNI152', '1mm')
('beliveau2017', 'sb207145', 'MNI152', '1mm')


{('beliveau2017',
  'az10419369',
  'MNI152',
  '1mm'): (<nibabel.gifti.gifti.GiftiImage at 0x2876535e0>, <nibabel.gifti.gifti.GiftiImage at 0x287685610>),
 ('beliveau2017',
  'cimbi36',
  'MNI152',
  '1mm'): (<nibabel.gifti.gifti.GiftiImage at 0x2876856d0>, <nibabel.gifti.gifti.GiftiImage at 0x287643f10>),
 ('beliveau2017',
  'cumi101',
  'MNI152',
  '1mm'): (<nibabel.gifti.gifti.GiftiImage at 0x2876989a0>, <nibabel.gifti.gifti.GiftiImage at 0x287698790>),
 ('beliveau2017',
  'dasb',
  'MNI152',
  '1mm'): (<nibabel.gifti.gifti.GiftiImage at 0x176200d30>, <nibabel.gifti.gifti.GiftiImage at 0x176200280>),
 ('beliveau2017',
  'sb207145',
  'MNI152',
  '1mm'): (<nibabel.gifti.gifti.GiftiImage at 0x2876af790>, <nibabel.gifti.gifti.GiftiImage at 0x284756880>)}

In [143]:
beliveau_parc = {}

for key, value in beliveau_fslr.items():
    if 'mni' in key[2].lower():  # Check if 'mni' (case-insensitive) is in the third element of the key tuple
        left_gii = value[0]  
        right_gii = value[1]  
        combined_gii = [left_gii, right_gii]
        transformed_values = glasser_parc.fit_transform(combined_gii, 'fsLR')
        beliveau_parc[key] = transformed_values


In [68]:
beliveau_parc.keys()

dict_keys([('beliveau2017', 'az10419369', 'MNI152', '1mm'), ('beliveau2017', 'cimbi36', 'MNI152', '1mm'), ('beliveau2017', 'cumi101', 'MNI152', '1mm'), ('beliveau2017', 'dasb', 'MNI152', '1mm'), ('beliveau2017', 'sb207145', 'MNI152', '1mm')])

In [188]:
compare_images(beliveau_parc[('beliveau2017', 'az10419369', 'MNI152', '1mm')], agg['regional_mean_ageeffect'], metric='pearsonr', nulls=rotated)

(-0.17899479678166227, 0.6853146853146853)

In [189]:
compare_images(beliveau_parc[('beliveau2017', 'cimbi36', 'MNI152', '1mm')], agg['regional_mean_ageeffect'], metric='pearsonr', nulls=rotated)

(-0.31782540710961277, 0.4065934065934066)

In [190]:
compare_images(beliveau_parc[('beliveau2017', 'cumi101', 'MNI152', '1mm')], agg['regional_mean_ageeffect'], metric='pearsonr', nulls=rotated)

(-0.47234702605614487, 0.13986013986013987)

In [191]:
compare_images(beliveau_parc[('beliveau2017', 'dasb', 'MNI152', '1mm')], agg['regional_mean_ageeffect'], metric='pearsonr', nulls=rotated)

(-0.36079526048057897, 0.32667332667332666)

In [192]:
compare_images(beliveau_parc[('beliveau2017', 'sb207145', 'MNI152', '1mm')], agg['regional_mean_ageeffect'], metric='pearsonr', nulls=rotated)

(-0.4074904770369426, 0.24875124875124874)

In [81]:
genes_fslr = transforms.fsaverage_to_fslr(genes, '32k')

left_gii = genes_fslr[0]  
right_gii = genes_fslr[1]  
combined_gii = [left_gii, right_gii]
transformed_values = glasser_parc.fit_transform(combined_gii, 'fsLR')
genes_parc = transformed_values

In [193]:
compare_images(genes_parc, agg['regional_mean_ageeffect'], metric='pearsonr', nulls=rotated)

(0.4651433382377202, 0.14685314685314685)

In [93]:
saaxis = fetch_annotation(source='sydnor2021', desc='SAaxis', space='fsLR', den='32k')


left_gii = saaxis[0]  
right_gii = saaxis[1]  
combined_gii = [left_gii, right_gii]
transformed_values = glasser_parc.fit_transform(combined_gii, 'fsLR')
saaxis_parc = transformed_values

In [194]:
compare_images(saaxis_parc, agg['regional_mean_ageeffect'], metric='pearsonr', nulls=rotated)

(-0.5105066361921242, 0.0959040959040959)

In [158]:
dukart = fetch_annotation(source='dukart2018', desc = 'flumazenil')
dukart_fslr = transforms.mni152_to_fslr(dukart, '32k')

left_gii = dukart_fslr[0]  
right_gii = dukart_fslr[1]  
combined_gii = [left_gii, right_gii]
transformed_values = glasser_parc.fit_transform(combined_gii, 'fsLR')
dukart_parc = transformed_values

In [195]:
compare_images(dukart_parc, agg['regional_mean_ageeffect'], metric='pearsonr', nulls=rotated)

(-0.3659618076383335, 0.3206793206793207)

In [165]:
norgaard = fetch_annotation(source='norgaard2021', desc = 'flumazenil', space = 'MNI152')
norgaard_fslr = transforms.mni152_to_fslr(norgaard, '32k')

left_gii = norgaard_fslr[0]  
right_gii = norgaard_fslr[1]  
combined_gii = [left_gii, right_gii]
transformed_values = glasser_parc.fit_transform(combined_gii, 'fsLR')
norgaard_parc = transformed_values

/Users/audluo/anaconda3/envs/luo_wm_dev/lib/python3.8/site-packages/neuromaps/datasets/annotations.py:238: UserWarning: Data from beliveau2017 and norgaard2021 is best used in the provided fsaverage space (e.g. source='beliveau2017', space='fsaverage', den='164k'). MNI152 maps should only be used for subcortical data.
  warnings.warn('Data from beliveau2017 and norgaard2021 is best used in'


In [196]:
compare_images(norgaard_parc, agg['regional_mean_ageeffect'], metric='pearsonr', nulls=rotated)

(0.035244021867483315, 0.942057942057942)